In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
# nakon što stavimo datoteke na Git uvijek čitati u ovomo obliku! (štedi vrijeme)
import time
start = time.time()
df = pd.read_hdf('./data/raw/training_dataset.h5', 'df')
#df = pd.read_hdf('./data/raw/nakon_procesiranja.h5', 'df')
end = time.time()
print(end - start)

# nakon što napravimo promjene kako treba, exportat ću ih u novu datoteku pomoću naredbe
# df.to_hdf('training_dataset2.h5', key='df')

63.18705892562866


In [16]:
# kasnije razmisliti o postavljanju na prosjek ovisno o vrsti klijenta? 
df['STAROST'].values[df['STAROST'].values >= 900] = -1
df['STAROST'].values[df['STAROST'].values < 0] = -1

In [4]:
df['PRIJEVREMENI_RASKID'] = np.where(df['DATUM_ZATVARANJA']>=(df['PLANIRANI_DATUM_ZATVARANJA']- np.timedelta64(10, 'D')), 'N', df['PRIJEVREMENI_RASKID'])

In [5]:
#brisanje outlier-a i ugovorenih iznosa = 0.0
df = df[(df['UGOVORENI_IZNOS'] < 2e5) & (df['UGOVORENI_IZNOS'] > 0.0)]

In [6]:
### BRISANJE PARTIJA S VIŠE KLIJENATA
start = time.time()
gb = df.groupby('OZNAKA_PARTIJE')['KLIJENT_ID'].nunique() # baci broj jedinstvenih klijenata za pojedinu partiju
# party_labels = df2['OZNAKA_PARTIJE'].unique().tolist() 
mask = gb[df['OZNAKA_PARTIJE']] == 1 # gb[df2['OZNAKA_PARTIJE']] vrati za svaku oznaku partije (svaki redak tablice) njenu vrijednost i usporedi je s 1
df = df[ mask.values ] # mask je neki objekt koji moram ovako koristiti, rezultat je skup gdje za svaku partiju ima jedan klijent
end = time.time()
print(end - start)

6.051986217498779


In [7]:
df = df[df['PLANIRANI_DATUM_ZATVARANJA'] > df['DATUM_OTVARANJA']] 

In [8]:
df.shape

(3830210, 17)

In [10]:
## POPUNJAVANJE MISSING VALUES UNAZAD (po grupama)

start = time.time()
# Create a groupby object
grouped = df.groupby(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA'])
end = time.time()
print( end - start)

# Write a function that imputes median
def impute_backwards(series):
    return series.fillna(method = 'bfill')

start = time.time()
# Impute age and assign to titanic['age']
df['NOVI_DATUM_ZATVARANJA']  = grouped['DATUM_ZATVARANJA'].transform(impute_backwards)
end = time.time()
print(end - start)

start = time.time()
# Impute age and assign to titanic['age']
df['NOVA_VISINA_KAMATE']  = grouped['VISINA_KAMATE'].transform(impute_backwards)
end = time.time()
print(end - start)

0.0
355.7915391921997
220.9151976108551


In [12]:
## POPUNJAVANJE MISSING VALUES UNAZAD (po grupama)

start = time.time()
# Create a groupby object
grouped = df.groupby(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA'])
end = time.time()
print( end - start)

# Write a function that imputes median
def impute_forwards(series):
    return series.fillna(method = 'ffill')

start = time.time()
# Impute age and assign to titanic['age']
df['NOVI_DATUM_ZATVARANJA2']  = grouped['NOVI_DATUM_ZATVARANJA'].transform(impute_forwards)
end = time.time()
print(end - start)

start = time.time()
# Impute age and assign to titanic['age']
df['NOVA_VISINA_KAMATE2']  = grouped['NOVA_VISINA_KAMATE'].transform(impute_forwards)
end = time.time()
print(end - start)

0.6879968643188477
355.42935848236084
221.71464681625366


In [13]:
missing_values = df.isnull().sum()
print(missing_values)

## pogledati jesu li uopće jedinstvene vrijednosti visine kamate u grupi, mozda se mijenjalo?

DATUM_IZVJESTAVANJA                     0
KLIJENT_ID                              0
OZNAKA_PARTIJE                          0
DATUM_OTVARANJA                         0
PLANIRANI_DATUM_ZATVARANJA              0
DATUM_ZATVARANJA                  2830145
UGOVORENI_IZNOS                         0
STANJE_NA_KRAJU_PRETH_KVARTALA          0
STANJE_NA_KRAJU_KVARTALA                0
VALUTA                                  0
VRSTA_KLIJENTA                          0
PROIZVOD                                0
VRSTA_PROIZVODA                         0
VISINA_KAMATE                       35417
TIP_KAMATE                              0
STAROST                                 0
PRIJEVREMENI_RASKID                     0
NOVI_DATUM_ZATVARANJA             1696065
NOVA_VISINA_KAMATE                   5584
NOVI_DATUM_ZATVARANJA2            1086755
NOVA_VISINA_KAMATE2                  1086
dtype: int64


In [11]:
df.shape

(3830210, 19)

In [16]:
n_unique = grouped['VISINA_KAMATE'].nunique()
print(n_unique)

KLIJENT_ID  OZNAKA_PARTIJE  DATUM_OTVARANJA
1063        7220374         2012-09-28         1
1100        7343568         2009-01-29         1
1103        1213645         2006-08-31         1
            5040752         2010-03-26         5
1108        5274589         2010-04-20         1
            5579344         2010-07-09         1
            7525231         2013-03-08         1
1112        5734885         2010-11-29         1
            5787834         2011-01-07         1
            5787836         2011-01-07         1
            5866535         2011-01-10         1
            5935821         2011-03-14         1
            6037245         2011-05-16         1
            6037246         2011-05-16         1
            6037247         2011-05-16         1
            6037248         2011-05-16         1
            6447309         2011-08-08         2
1115        6729753         2012-02-10         1
            7151338         2012-08-13         1
            7483023      

In [17]:
def impute_mean(series):
    return series.fillna(series.mean())


grouped = df.groupby(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA'])

start = time.time()
df['AVG_VISINA_KAMATE']  = grouped['VISINA_KAMATE'].transform(impute_mean)
end = time.time()
print(end - start)

295.0074396133423


In [19]:
print(df.isnull().sum())

DATUM_IZVJESTAVANJA                     0
KLIJENT_ID                              0
OZNAKA_PARTIJE                          0
DATUM_OTVARANJA                         0
PLANIRANI_DATUM_ZATVARANJA              0
DATUM_ZATVARANJA                  2830145
UGOVORENI_IZNOS                         0
STANJE_NA_KRAJU_PRETH_KVARTALA          0
STANJE_NA_KRAJU_KVARTALA                0
VALUTA                                  0
VRSTA_KLIJENTA                          0
PROIZVOD                                0
VRSTA_PROIZVODA                         0
VISINA_KAMATE                       35417
TIP_KAMATE                              0
STAROST                                 0
PRIJEVREMENI_RASKID                     0
NOVI_DATUM_ZATVARANJA             1696065
NOVA_VISINA_KAMATE                   5584
NOVI_DATUM_ZATVARANJA2            1086755
NOVA_VISINA_KAMATE2                  1086
AVG_VISINA_KAMATE                    1086
dtype: int64


In [21]:
# Write a function that imputes median
def impute_backwards(series):
    return series.fillna(method = 'bfill')

prijevremeni_raskidi = df[df['PRIJEVREMENI_RASKID'] == 'Y']
group_raskidi = prijevremeni_raskidi.groupby(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA'])

start = time.time()
# Impute age and assign to titanic['age']
prijevremeni_raskidi['NOVI_DATUM_ZATVARANJA']  = group_raskidi['DATUM_ZATVARANJA'].transform(impute_backwards)
end = time.time()
print(end - start)

75.03388452529907


C:\Users\Jakob\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
# neka tu stoji ako zatreba
# sortiranje, potrebno za filanje missing values
# df.sort_values(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA', 'DATUM_IZVJESTAVANJA', 'STANJE_NA_KRAJU_KVARTALA'], ascending=[True, True, True, True, False], inplace = True)

In [ ]:
# export datoteke
df.to_hdf('./data/raw/nakon_procesiranja.h5', key='df')

In [22]:
df.shape

(3830210, 22)

In [23]:
df.columns

Index(['DATUM_IZVJESTAVANJA', 'KLIJENT_ID', 'OZNAKA_PARTIJE',
       'DATUM_OTVARANJA', 'PLANIRANI_DATUM_ZATVARANJA', 'DATUM_ZATVARANJA',
       'UGOVORENI_IZNOS', 'STANJE_NA_KRAJU_PRETH_KVARTALA',
       'STANJE_NA_KRAJU_KVARTALA', 'VALUTA', 'VRSTA_KLIJENTA', 'PROIZVOD',
       'VRSTA_PROIZVODA', 'VISINA_KAMATE', 'TIP_KAMATE', 'STAROST',
       'PRIJEVREMENI_RASKID', 'NOVI_DATUM_ZATVARANJA', 'NOVA_VISINA_KAMATE',
       'NOVI_DATUM_ZATVARANJA2', 'NOVA_VISINA_KAMATE2', 'AVG_VISINA_KAMATE'],
      dtype='object')

In [25]:
df_copy = df.copy()
del_col_list = ['DATUM_IZVJESTAVANJA','STANJE_NA_KRAJU_PRETH_KVARTALA', 'STANJE_NA_KRAJU_KVARTALA', 'DATUM_ZATVARANJA', 'NOVI_DATUM_ZATVARANJA', 'VISINA_KAMATE', 'NOVA_VISINA_KAMATE', 'NOVA_VISINA_KAMATE2']
df_copy.drop(del_col_list, axis = 1, inplace = True)
df_copy.columns

Index(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA',
       'PLANIRANI_DATUM_ZATVARANJA', 'UGOVORENI_IZNOS', 'VALUTA',
       'VRSTA_KLIJENTA', 'PROIZVOD', 'VRSTA_PROIZVODA', 'TIP_KAMATE',
       'STAROST', 'PRIJEVREMENI_RASKID', 'NOVI_DATUM_ZATVARANJA2',
       'AVG_VISINA_KAMATE'],
      dtype='object')

In [28]:
print(df_copy.isnull().sum())

KLIJENT_ID                          0
OZNAKA_PARTIJE                      0
DATUM_OTVARANJA                     0
PLANIRANI_DATUM_ZATVARANJA          0
UGOVORENI_IZNOS                     0
VALUTA                              0
VRSTA_KLIJENTA                      0
PROIZVOD                            0
VRSTA_PROIZVODA                     0
TIP_KAMATE                          0
STAROST                             0
PRIJEVREMENI_RASKID                 0
NOVI_DATUM_ZATVARANJA2        1086755
AVG_VISINA_KAMATE                1086
dtype: int64


In [36]:
raskidi = df_copy[df_copy['PRIJEVREMENI_RASKID'] == 'Y']

In [32]:
print(raskidi.isnull().sum())

KLIJENT_ID                      0
OZNAKA_PARTIJE                  0
DATUM_OTVARANJA                 0
PLANIRANI_DATUM_ZATVARANJA      0
UGOVORENI_IZNOS                 0
VALUTA                          0
VRSTA_KLIJENTA                  0
PROIZVOD                        0
VRSTA_PROIZVODA                 0
TIP_KAMATE                      0
STAROST                         0
PRIJEVREMENI_RASKID             0
NOVI_DATUM_ZATVARANJA2          0
AVG_VISINA_KAMATE             247
dtype: int64


In [38]:
raskidi.dropna(subset = ['AVG_VISINA_KAMATE'], inplace = True)
raskidi.shape

C:\Users\Jakob\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(266454, 14)

In [39]:
df_copy.dropna(subset = ['AVG_VISINA_KAMATE'], inplace = True)

In [40]:
df_copy.shape

(3829124, 14)

In [50]:
df_copy['NOVI_DATUM_ZATVARANJA2'] = np.where(df_copy['NOVI_DATUM_ZATVARANJA2'].isnull(), df_copy['PLANIRANI_DATUM_ZATVARANJA'] , df_copy['NOVI_DATUM_ZATVARANJA2'])

In [53]:
df_copy.columns

Index(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA',
       'PLANIRANI_DATUM_ZATVARANJA', 'UGOVORENI_IZNOS', 'VALUTA',
       'VRSTA_KLIJENTA', 'PROIZVOD', 'VRSTA_PROIZVODA', 'TIP_KAMATE',
       'STAROST', 'PRIJEVREMENI_RASKID', 'NOVI_DATUM_ZATVARANJA2',
       'AVG_VISINA_KAMATE'],
      dtype='object')

In [54]:
df_copy.drop_duplicates(subset = ['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA'], inplace = True)

In [55]:
df_copy.shape

(673448, 14)

In [56]:
df_copy.head()

,KLIJENT_ID,OZNAKA_PARTIJE,DATUM_OTVARANJA,PLANIRANI_DATUM_ZATVARANJA,UGOVORENI_IZNOS,VALUTA,VRSTA_KLIJENTA,PROIZVOD,VRSTA_PROIZVODA,TIP_KAMATE,STAROST,PRIJEVREMENI_RASKID,NOVI_DATUM_ZATVARANJA2,AVG_VISINA_KAMATE
1,772139,5521579,2010-06-11,2017-06-30,46000.00,1,1410,FL0801,A,B,38,N,2017-02-16,9.70
2,481985,7443823,2013-01-23,2018-02-28,44679.50,2,1550,FD0100,L,A,37,N,2015-09-07,3.10
3,139014,6511639,2011-09-16,2016-09-30,29959.15,2,1410,FL0801,A,B,49,N,2014-06-16,7.95
4,952566,6143123,2011-05-23,2014-05-31,100000.00,1,1410,FL0801,A,B,35,N,2014-04-10,9.70
6,686960,7791434,2013-05-27,2015-05-31,4017.00,1,1410,FL1100,A,A,65,N,2014-10-16,11.95


In [59]:
# duljina kredita je dana u danima
df_copy['PLANIRANA_DULJINA_KREDITA'] = (df_copy['PLANIRANI_DATUM_ZATVARANJA']-df_copy['DATUM_OTVARANJA']).dt.days

In [61]:
del df_copy['PLANIRANI_DATUM_ZATVARANJA']

In [62]:
df_copy.head()

,KLIJENT_ID,OZNAKA_PARTIJE,DATUM_OTVARANJA,UGOVORENI_IZNOS,VALUTA,VRSTA_KLIJENTA,PROIZVOD,VRSTA_PROIZVODA,TIP_KAMATE,STAROST,PRIJEVREMENI_RASKID,NOVI_DATUM_ZATVARANJA2,AVG_VISINA_KAMATE,PLANIRANA_DULJINA_KREDITA
1,772139,5521579,2010-06-11,46000.00,1,1410,FL0801,A,B,38,N,2017-02-16,9.70,2576
2,481985,7443823,2013-01-23,44679.50,2,1550,FD0100,L,A,37,N,2015-09-07,3.10,1862
3,139014,6511639,2011-09-16,29959.15,2,1410,FL0801,A,B,49,N,2014-06-16,7.95,1841
4,952566,6143123,2011-05-23,100000.00,1,1410,FL0801,A,B,35,N,2014-04-10,9.70,1104
6,686960,7791434,2013-05-27,4017.00,1,1410,FL1100,A,A,65,N,2014-10-16,11.95,734


In [68]:
# sluzi za merganje s makroekonomskim indikatorima
df_copy['GODINA_OTVARANJA']=df_copy['DATUM_OTVARANJA'].dt.year
df_copy['GODINA_ZATVARANJA']=df_copy['NOVI_DATUM_ZATVARANJA2'].dt.year

<h1>Makroekonomski indikatori</h1>

In [74]:
# Čitam tablicu ekonomskih indikatora
econ = pd.read_excel('./data/external/ekonomski_indikatori.xlsx')

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,NaN,General information on Croatia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Economic Indicators,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,2000,2001,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,...,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018


In [75]:
econ.dropna(thresh=int(econ.shape[1] * .9), axis = 0, inplace = True)
column_names = econ.columns
# Drop unwanted columns
econ.drop(column_names[0], inplace=True, axis=1)

In [77]:
econ.shape
econ.head()

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
4,NaN,2000,2001,2002.000000,2003.000000,2004.000000,2005.000000,2006.000000,2007.000000,2008.000000,2009.000000,2010.000000,2011.000000,2012.000000,2013.000000,2014.000000,2015.000000,2016.000000,2017.000000,2018
5,Area (square km),56594,56594,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594
6,Population (million) a,4.426,4.30549,4.305384,4.305725,4.310861,4.312487,4.313530,4.311967,4.309796,4.302847,4.289857,4.280000,4.268000,4.256000,4.238000,4.204000,4.174000,4.125000,4.089
7,"GDP (million HRK, current prices) b",180241,194271,211500.384280,232444.050150,250564.762188,269779.451979,294150.258439,322463.846521,347749.878891,331367.071951,329143.142619,333456.571836,330824.955608,331784.893258,331569.648134,339616.204419,351349.058619,365643.149141,381799
8,"GDP (million EUR, current prices)",23610.7,26003.3,28554.214875,30729.300540,33427.889423,36456.451152,40168.827520,43956.244732,48143.611980,45148.120983,45173.312209,44854.374703,44008.247014,43808.383238,43455.968513,44629.962125,46663.725118,49013.169950,51496.3


In [78]:
econ.rename(columns={'Unnamed: 2': '2000', 'Unnamed: 3': '2001', 
                   'Unnamed: 4': '2002', 'Unnamed: 5': '2003', 
                   'Unnamed: 6': '2004', 'Unnamed: 7': '2005',
                  'Unnamed: 8': '2006', 'Unnamed: 9': '2007', 
                  'Unnamed: 10': '2008', 'Unnamed: 11': '2009',
                  'Unnamed: 12': '2010', 'Unnamed: 13': '2011',
                  'Unnamed: 14': '2012', 'Unnamed: 15': '2013',
                  'Unnamed: 16': '2014', 'Unnamed: 17': '2015',
                  'Unnamed: 18': '2016', 'Unnamed: 19': '2017',
                  'Unnamed: 20': '2018'}, inplace=True)

In [79]:
econ.head()

,Unnamed: 1,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
4,NaN,2000,2001,2002.000000,2003.000000,2004.000000,2005.000000,2006.000000,2007.000000,2008.000000,2009.000000,2010.000000,2011.000000,2012.000000,2013.000000,2014.000000,2015.000000,2016.000000,2017.000000,2018
5,Area (square km),56594,56594,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594
6,Population (million) a,4.426,4.30549,4.305384,4.305725,4.310861,4.312487,4.313530,4.311967,4.309796,4.302847,4.289857,4.280000,4.268000,4.256000,4.238000,4.204000,4.174000,4.125000,4.089
7,"GDP (million HRK, current prices) b",180241,194271,211500.384280,232444.050150,250564.762188,269779.451979,294150.258439,322463.846521,347749.878891,331367.071951,329143.142619,333456.571836,330824.955608,331784.893258,331569.648134,339616.204419,351349.058619,365643.149141,381799
8,"GDP (million EUR, current prices)",23610.7,26003.3,28554.214875,30729.300540,33427.889423,36456.451152,40168.827520,43956.244732,48143.611980,45148.120983,45173.312209,44854.374703,44008.247014,43808.383238,43455.968513,44629.962125,46663.725118,49013.169950,51496.3


In [80]:
econ.drop([4], axis = 0, inplace = True)

In [81]:
econ.head()

,Unnamed: 1,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
5,Area (square km),56594,56594,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594.000000,56594
6,Population (million) a,4.426,4.30549,4.305384,4.305725,4.310861,4.312487,4.313530,4.311967,4.309796,4.302847,4.289857,4.280000,4.268000,4.256000,4.238000,4.204000,4.174000,4.125000,4.089
7,"GDP (million HRK, current prices) b",180241,194271,211500.384280,232444.050150,250564.762188,269779.451979,294150.258439,322463.846521,347749.878891,331367.071951,329143.142619,333456.571836,330824.955608,331784.893258,331569.648134,339616.204419,351349.058619,365643.149141,381799
8,"GDP (million EUR, current prices)",23610.7,26003.3,28554.214875,30729.300540,33427.889423,36456.451152,40168.827520,43956.244732,48143.611980,45148.120983,45173.312209,44854.374703,44008.247014,43808.383238,43455.968513,44629.962125,46663.725118,49013.169950,51496.3
9,GDP per capita (in EUR),5334.56,6039.56,6632.210942,7136.846998,7754.341748,8453.695316,9312.286577,10194.012322,11170.740327,10492.615931,10530.260615,10479.994090,10311.210641,10293.323129,10253.885916,10616.070915,11179.617901,11881.980594,12593.9


In [ ]:
econ2 = econ.transpose(copy = True)
new_column_names = econ2.iloc[0,:]
econ2.columns = new_column_names

In [84]:
econ2.reset_index()

Unnamed: 1,index,Area (square km),Population (million) a,"GDP (million HRK, current prices) b","GDP (million EUR, current prices)",GDP per capita (in EUR),"GDP - year-on-year rate of growth (in %, constant prices)",Average CPI year-on-year inflation rate,Current account balance (million EUR) c,Current account balance (as of % GDP),...,"Gross international reserves (in terms of months of imports of goods and services, end of year)",Exchange rate on 31 December (HRK : 1 EUR),Exchange rate on 31 December (HRK : 1 USD),Average exchange rate (HRK : 1 EUR),Average exchange rate (HRK : 1 USD),Consolidated general government net lending (+)/borrowing (-) (million HRK) e,Consolidated general government net lending (+)/borrowing (-) (as % of GDP) e,General government debt (as % of GDP) e,"Unemployment rate (ILO, persons above 15 years of age) f","Employment rate (ILO, persons above 15 years of age) f"
0,Unnamed: 1,Area (square km),Population (million) a,"GDP (million HRK, current prices) b","GDP (million EUR, current prices)",GDP per capita (in EUR),"GDP - year-on-year rate of growth (in %, const...",Average CPI year-on-year inflation rate,Current account balance (million EUR) c,Current account balance (as of % GDP),...,Gross international reserves (in terms of mont...,Exchange rate on 31 December (HRK : 1 EUR),Exchange rate on 31 December (HRK : 1 USD),Average exchange rate (HRK : 1 EUR),Average exchange rate (HRK : 1 USD),Consolidated general government net lending (+...,Consolidated general government net lending (+...,General government debt (as % of GDP) e,"Unemployment rate (ILO, persons above 15 years...","Employment rate (ILO, persons above 15 years o..."
1,2000,56594,4.426,180241,23610.7,5334.56,3.7675,4.6,-516.727,-2.18852,...,4.85331,7.59833,8.15534,7.63385,8.28737,....,....,35.4986,16.05,42.6
2,2001,56594,4.30549,194271,26003.3,6039.56,3.45241,3.8,-758.857,-2.91831,...,5.81721,7.37003,8.35604,7.47101,8.33915,....,....,36.4623,15.8,41.8
3,2002,56594,4.30538,211500,28554.2,6632.21,5.25336,1.7,-2016.88,-7.06332,...,5.20306,7.44229,7.14574,7.40698,7.87249,-7365.19,-3.48235,36.6016,14.8,43.3
4,2003,56594,4.30572,232444,30729.3,7136.85,5.58119,1.8,-1811.73,-5.89579,...,5.52068,7.64691,6.11851,7.56425,6.70445,-10863.8,-4.67371,38.0835,14.25,43.1
5,2004,56594,4.31086,250565,33427.9,7754.34,3.91456,2.1,-1355.35,-4.05455,...,5.0642,7.67123,5.63688,7.49568,6.03122,-13038.5,-5.20365,40.2681,13.8,43.5
6,2005,56594,4.31249,269779,36456.5,8453.7,4.10956,3.3,-1892.06,-5.18991,...,5.37051,7.37563,6.23363,7.40005,5.94996,-10413.5,-3.86001,41.1553,12.7,43.3
7,2006,56594,4.31353,294150,40168.8,9312.29,4.86782,3.2,-2612.7,-6.5043,...,5.60497,7.34508,5.5784,7.32285,5.83917,-9971.92,-3.39008,38.6436,11.2,43.6
8,2007,56594,4.31197,322464,43956.2,10194,5.28365,2.9,-3138.26,-7.13952,...,5.48831,7.32513,4.98546,7.33602,5.36599,-7880.62,-2.44388,37.2272,9.9,47.6
9,2008,56594,4.3098,347750,48143.6,11170.7,2.03646,6.1,-4226.91,-8.7798,...,4.88824,7.32442,5.1555,7.22318,4.93442,-9604.64,-2.76194,38.9616,8.5,48.6


In [115]:
#econ2.drop([0], axis = 0, inplace = True)
indikatori = econ2.iloc[1:, :]
indikatori['GODINE']=range(2000,2019)

C:\Users\Jakob\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [116]:
indikatori.head(19)

Unnamed: 1,Area (square km),Population (million) a,"GDP (million HRK, current prices) b","GDP (million EUR, current prices)",GDP per capita (in EUR),"GDP - year-on-year rate of growth (in %, constant prices)",Average CPI year-on-year inflation rate,Current account balance (million EUR) c,Current account balance (as of % GDP),Exports of goods and services (as of % GDP),...,Exchange rate on 31 December (HRK : 1 EUR),Exchange rate on 31 December (HRK : 1 USD),Average exchange rate (HRK : 1 EUR),Average exchange rate (HRK : 1 USD),Consolidated general government net lending (+)/borrowing (-) (million HRK) e,Consolidated general government net lending (+)/borrowing (-) (as % of GDP) e,General government debt (as % of GDP) e,"Unemployment rate (ILO, persons above 15 years of age) f","Employment rate (ILO, persons above 15 years of age) f",GODINE
2000,56594,4.426,180241,23610.7,5334.56,3.7675,4.6,-516.727,-2.18852,35.215,...,7.59833,8.15534,7.63385,8.28737,....,....,35.4986,16.05,42.6,2000
2001,56594,4.30549,194271,26003.3,6039.56,3.45241,3.8,-758.857,-2.91831,37.2483,...,7.37003,8.35604,7.47101,8.33915,....,....,36.4623,15.8,41.8,2001
2002,56594,4.30538,211500,28554.2,6632.21,5.25336,1.7,-2016.88,-7.06332,36.24,...,7.44229,7.14574,7.40698,7.87249,-7365.19,-3.48235,36.6016,14.8,43.3,2002
2003,56594,4.30572,232444,30729.3,7136.85,5.58119,1.8,-1811.73,-5.89579,39.7021,...,7.64691,6.11851,7.56425,6.70445,-10863.8,-4.67371,38.0835,14.25,43.1,2003
2004,56594,4.31086,250565,33427.9,7754.34,3.91456,2.1,-1355.35,-4.05455,39.7148,...,7.67123,5.63688,7.49568,6.03122,-13038.5,-5.20365,40.2681,13.8,43.5,2004
2005,56594,4.31249,269779,36456.5,8453.7,4.10956,3.3,-1892.06,-5.18991,39.4753,...,7.37563,6.23363,7.40005,5.94996,-10413.5,-3.86001,41.1553,12.7,43.3,2005
2006,56594,4.31353,294150,40168.8,9312.29,4.86782,3.2,-2612.7,-6.5043,39.771,...,7.34508,5.5784,7.32285,5.83917,-9971.92,-3.39008,38.6436,11.2,43.6,2006
2007,56594,4.31197,322464,43956.2,10194,5.28365,2.9,-3138.26,-7.13952,39.0262,...,7.32513,4.98546,7.33602,5.36599,-7880.62,-2.44388,37.2272,9.9,47.6,2007
2008,56594,4.3098,347750,48143.6,11170.7,2.03646,6.1,-4226.91,-8.7798,38.4784,...,7.32442,5.1555,7.22318,4.93442,-9604.64,-2.76194,38.9616,8.5,48.6,2008
2009,56594,4.30285,331367,45148.1,10492.6,-7.29143,2.4,-2299.25,-5.09269,34.5037,...,7.3062,5.0893,7.33955,5.28037,-19844,-5.98853,48.2533,9.2,48.2,2009


In [117]:
indikatori.columns

Index(['Area (square km)', 'Population (million) a',
       'GDP (million HRK, current prices) b',
       'GDP (million EUR, current prices) ', 'GDP per capita (in EUR)  ',
       'GDP - year-on-year rate of growth (in %, constant prices) ',
       'Average CPI year-on-year inflation rate ',
       'Current account balance (million EUR) c',
       'Current account balance (as of % GDP) ',
       'Exports of goods and services (as of % GDP) ',
       'Imports of goods and services (as of % GDP) ',
       'External debt (million EUR, end of year) c',
       'External debt (as of % GDP) ',
       'External debt (as of % exports of goods and services) ',
       'External debt service (as % of exports of goods and services) d',
       'Gross international reserves (million EUR, end of year)',
       'Gross international reserves (in terms of months of imports of goods and services, end of year)',
       'Exchange rate on 31 December (HRK : 1 EUR)',
       'Exchange rate on 31 December  (HRK

In [118]:
drop_col = [0,1,2, 3,7,11, 13, 14, 15, 16, 17, 18, 24]
indikatori.columns[drop_col].values
indikatori.drop(indikatori.columns[drop_col].values, axis = 1, inplace = True)

In [120]:
indikatori.head()

Unnamed: 1,GDP per capita (in EUR),"GDP - year-on-year rate of growth (in %, constant prices)",Average CPI year-on-year inflation rate,Current account balance (as of % GDP),Exports of goods and services (as of % GDP),Imports of goods and services (as of % GDP),External debt (as of % GDP),Average exchange rate (HRK : 1 EUR),Average exchange rate (HRK : 1 USD),Consolidated general government net lending (+)/borrowing (-) (million HRK) e,Consolidated general government net lending (+)/borrowing (-) (as % of GDP) e,General government debt (as % of GDP) e,"Employment rate (ILO, persons above 15 years of age) f",GODINE
2000,5334.56,3.7675,4.6,-2.18852,35.215,39.6182,51.9406,7.63385,8.28737,....,....,35.4986,42.6,2000
2001,6039.56,3.45241,3.8,-2.91831,37.2483,42.3117,52.3366,7.47101,8.33915,....,....,36.4623,41.8,2001
2002,6632.21,5.25336,1.7,-7.06332,36.24,45.6459,53.036,7.40698,7.87249,-7365.19,-3.48235,36.6016,43.3,2002
2003,7136.85,5.58119,1.8,-5.89579,39.7021,46.3605,64.7067,7.56425,6.70445,-10863.8,-4.67371,38.0835,43.1,2003
2004,7754.34,3.91456,2.1,-4.05455,39.7148,45.6236,68.6055,7.49568,6.03122,-13038.5,-5.20365,40.2681,43.5,2004


In [123]:
df_copy.columns

Index(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA', 'UGOVORENI_IZNOS',
       'VALUTA', 'VRSTA_KLIJENTA', 'PROIZVOD', 'VRSTA_PROIZVODA', 'TIP_KAMATE',
       'STAROST', 'PRIJEVREMENI_RASKID', 'NOVI_DATUM_ZATVARANJA2',
       'AVG_VISINA_KAMATE', 'PLANIRANA_DULJINA_KREDITA', 'GODINA_OTVARANJA',
       'GODINA_ZATVARANJA'],
      dtype='object')

In [131]:
indikatori.columns

Index(['GDP per capita (in EUR)  ',
       'GDP - year-on-year rate of growth (in %, constant prices) ',
       'Average CPI year-on-year inflation rate ',
       'Current account balance (as of % GDP) ',
       'Exports of goods and services (as of % GDP) ',
       'Imports of goods and services (as of % GDP) ',
       'External debt (as of % GDP) ', 'Average exchange rate (HRK : 1 EUR) ',
       'Average exchange rate (HRK : 1 USD) ',
       'Consolidated general government net lending (+)/borrowing (-) (million HRK) e',
       'Consolidated general government net lending (+)/borrowing (-) (as % of GDP) e',
       'General government debt (as % of GDP) e',
       'Employment rate (ILO, persons above 15 years of age) f', 'GODINE'],
      dtype='object', name='Unnamed: 1')

In [152]:
start = time.time()
df_merged = pd.merge(df_copy, indikatori, left_on=['GODINA_OTVARANJA'], right_on = ['GODINE'], how = 'left')
end = time.time()
print(end-start)
df_merged.rename(columns={'GDP per capita (in EUR)  ': 'ind1-poc', 'GDP - year-on-year rate of growth (in %, constant prices) ': 'ind2-poc', 
                   'Average CPI year-on-year inflation rate ': 'ind3-poc', 'Current account balance (as of % GDP) ': 'ind4-poc', 
                   'Exports of goods and services (as of % GDP) ': 'ind5-poc', 'Imports of goods and services (as of % GDP) ': 'ind6-poc',
                  'External debt (as of % GDP) ': 'ind7-poc', 'Average exchange rate (HRK : 1 EUR) ': 'ind8-poc', 'Average exchange rate (HRK : 1 USD) ': 'ind9-poc',
                  'Consolidated general government net lending (+)/borrowing (-) (as % of GDP) e' : 'ind10-poc', 'General government debt (as % of GDP) e': 'ind11-poc',
                  'Employment rate (ILO, persons above 15 years of age) f': 'ind12-poc'}, inplace=True)

0.48877573013305664


In [153]:
del df_merged['Consolidated general government net lending (+)/borrowing (-) (million HRK) e']
del df_merged['GODINE']

In [154]:
df_merged.head()

,KLIJENT_ID,OZNAKA_PARTIJE,DATUM_OTVARANJA,UGOVORENI_IZNOS,VALUTA,VRSTA_KLIJENTA,PROIZVOD,VRSTA_PROIZVODA,TIP_KAMATE,STAROST,...,ind3-poc,ind4-poc,ind5-poc,ind6-poc,ind7-poc,ind8-poc,ind9-poc,ind10-poc,ind11-poc,ind12-poc
0,772139,5521579,2010-06-11,46000.00,1,1410,FL0801,A,B,38,...,1.1,-1.0676,37.6476,37.982,103.841,7.28623,5.50002,-6.27526,57.3055,46.5
1,481985,7443823,2013-01-23,44679.50,2,1550,FD0100,L,A,37,...,2.2,0.946094,42.8402,42.4593,104.554,7.57355,5.70588,-5.2795,80.3602,42.075
2,139014,6511639,2011-09-16,29959.15,2,1410,FL0801,A,B,49,...,2.3,-0.698502,40.415,40.8333,103.44,7.4342,5.34351,-7.86303,63.8248,44.8
3,952566,6143123,2011-05-23,100000.00,1,1410,FL0801,A,B,35,...,2.3,-0.698502,40.415,40.8333,103.44,7.4342,5.34351,-7.86303,63.8248,44.8
4,686960,7791434,2013-05-27,4017.00,1,1410,FL1100,A,A,65,...,2.2,0.946094,42.8402,42.4593,104.554,7.57355,5.70588,-5.2795,80.3602,42.075


In [155]:
start = time.time()
df_merged2 = pd.merge(df_merged, indikatori, left_on=['GODINA_ZATVARANJA'], right_on = ['GODINE'], how = 'left')
end = time.time()
print(end-start)

df_merged2.rename(columns={'GDP per capita (in EUR)  ': 'ind1-zav', 'GDP - year-on-year rate of growth (in %, constant prices) ': 'ind2-zav', 
                   'Average CPI year-on-year inflation rate ': 'ind3-zav', 'Current account balance (as of % GDP) ': 'ind4-zav', 
                   'Exports of goods and services (as of % GDP) ': 'ind5-zav', 'Imports of goods and services (as of % GDP) ': 'ind6-zav',
                  'External debt (as of % GDP) ': 'ind7-zav', 'Average exchange rate (HRK : 1 EUR) ': 'ind8-zav', 'Average exchange rate (HRK : 1 USD) ': 'ind9-zav',
                  'Consolidated general government net lending (+)/borrowing (-) (as % of GDP) e' : 'ind10-zav', 'General government debt (as % of GDP) e': 'ind11-zav',
                  'Employment rate (ILO, persons above 15 years of age) f': 'ind12-zav'}, inplace=True)


0.8716411590576172


In [156]:
del df_merged2['Consolidated general government net lending (+)/borrowing (-) (million HRK) e']
del df_merged2['GODINE']
df_merged2.columns

Index(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA', 'UGOVORENI_IZNOS',
       'VALUTA', 'VRSTA_KLIJENTA', 'PROIZVOD', 'VRSTA_PROIZVODA', 'TIP_KAMATE',
       'STAROST', 'PRIJEVREMENI_RASKID', 'NOVI_DATUM_ZATVARANJA2',
       'AVG_VISINA_KAMATE', 'PLANIRANA_DULJINA_KREDITA', 'GODINA_OTVARANJA',
       'GODINA_ZATVARANJA', 'ind1-poc', 'ind2-poc', 'ind3-poc', 'ind4-poc',
       'ind5-poc', 'ind6-poc', 'ind7-poc', 'ind8-poc', 'ind9-poc', 'ind10-poc',
       'ind11-poc', 'ind12-poc', 'ind1-zav', 'ind2-zav', 'ind3-zav',
       'ind4-zav', 'ind5-zav', 'ind6-zav', 'ind7-zav', 'ind8-zav', 'ind9-zav',
       'ind10-zav', 'ind11-zav', 'ind12-zav'],
      dtype='object')

In [157]:
df_merged.shape

(673448, 28)

In [158]:
df_merged2.shape

(673448, 40)

In [159]:
df_copy.shape

(673448, 16)

<h2> Nakon upload-a procesirane datoteke </h2>

In [6]:
df.shape

(3830210, 22)

In [7]:
df.head()

,DATUM_IZVJESTAVANJA,KLIJENT_ID,OZNAKA_PARTIJE,DATUM_OTVARANJA,PLANIRANI_DATUM_ZATVARANJA,DATUM_ZATVARANJA,UGOVORENI_IZNOS,STANJE_NA_KRAJU_PRETH_KVARTALA,STANJE_NA_KRAJU_KVARTALA,VALUTA,...,VRSTA_PROIZVODA,VISINA_KAMATE,TIP_KAMATE,STAROST,PRIJEVREMENI_RASKID,NOVI_DATUM_ZATVARANJA,NOVA_VISINA_KAMATE,NOVI_DATUM_ZATVARANJA2,NOVA_VISINA_KAMATE2,AVG_VISINA_KAMATE
1,2013-12-31,772139,5521579,2010-06-11,2017-06-30,NaT,46000.00,28481.94,26887.19,1,...,A,9.70,B,38,N,2017-02-16,9.70,2017-02-16,9.70,9.70
2,2013-12-31,481985,7443823,2013-01-23,2018-02-28,NaT,44679.50,44908.40,45062.09,2,...,L,3.10,A,37,N,2015-09-07,3.10,2015-09-07,3.10,3.10
3,2013-12-31,139014,6511639,2011-09-16,2016-09-30,NaT,29959.15,19691.64,18285.89,2,...,A,7.95,B,49,N,2014-06-16,7.95,2014-06-16,7.95,7.95
4,2013-12-31,952566,6143123,2011-05-23,2014-05-31,NaT,100000.00,24797.93,15685.33,1,...,A,9.70,B,35,N,2014-04-10,9.70,2014-04-10,9.70,9.70
6,2013-12-31,686960,7791434,2013-05-27,2015-05-31,NaT,4017.00,3412.05,2942.34,1,...,A,11.95,A,65,N,2014-10-16,11.95,2014-10-16,11.95,11.95


In [8]:
df.isnull().sum()

DATUM_IZVJESTAVANJA                     0
KLIJENT_ID                              0
OZNAKA_PARTIJE                          0
DATUM_OTVARANJA                         0
PLANIRANI_DATUM_ZATVARANJA              0
DATUM_ZATVARANJA                  2830145
UGOVORENI_IZNOS                         0
STANJE_NA_KRAJU_PRETH_KVARTALA          0
STANJE_NA_KRAJU_KVARTALA                0
VALUTA                                  0
VRSTA_KLIJENTA                          0
PROIZVOD                                0
VRSTA_PROIZVODA                         0
VISINA_KAMATE                       35417
TIP_KAMATE                              0
STAROST                                 0
PRIJEVREMENI_RASKID                     0
NOVI_DATUM_ZATVARANJA             1696065
NOVA_VISINA_KAMATE                   5584
NOVI_DATUM_ZATVARANJA2            1086755
NOVA_VISINA_KAMATE2                  1086
AVG_VISINA_KAMATE                    1086
dtype: int64

In [9]:
del df['DATUM_ZATVARANJA']

In [10]:
del df['VISINA_KAMATE']

In [11]:
del df['NOVI_DATUM_ZATVARANJA']

In [12]:
del df['NOVA_VISINA_KAMATE']

In [13]:
df.shape

(3830210, 18)